In [1]:
import sqlite3
import pandas as pd
from tabulate import tabulate

In [2]:
import os

def generate_new_database_filename():
    base_name = "database"
    counter = 1

    # Increment counter until a new file name is found
    while os.path.exists(f"{base_name}{counter}.db"):
        counter += 1

    return f"{base_name}{counter}.db"

name = generate_new_database_filename()
conn = sqlite3.connect(name)

In [3]:
cursor = conn.cursor()

In [4]:
tables = cursor.fetchall()

# drop all tables in the database
for table in tables:
    cursor.execute(f"DROP TABLE {table[0]}")
conn.commit()
    

In [5]:
# Teatersal
cursor.executescript(
    """
CREATE TABLE Teatersal (
    Navn VARCHAR(255) PRIMARY KEY NOT NULL,
    Kapasitet INT
);
CREATE TABLE Theaterstykke (Navn VARCHAR(255) PRIMARY KEY NOT NULL);
CREATE TABLE Oppgave (Beskrivelse TEXT PRIMARY KEY NOT NULL);
CREATE TABLE Oppstilling (
    SesongId INTEGER NOT NULL,
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    SalNavn VARCHAR(255) NOT NULL,
    PRIMARY KEY (SesongId, TeaterStykkeNavn, SalNavn),
    FOREIGN KEY (SesongId) REFERENCES Sesong(SesongId) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (SalNavn) REFERENCES Teatersal(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Ansatt (
    AnsattId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    Epost VARCHAR(255),
    Navn VARCHAR(255),
    Ansattstatus VARCHAR(255),
    ErSjef BOOLEAN DEFAULT FALSE
);
CREATE TABLE Kundeprofil (
    Mobilnummer VARCHAR(20) PRIMARY KEY NOT NULL,
    Navn VARCHAR(255),
    Adresse TEXT
);
CREATE TABLE Sesong (
    SesongId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    Navn VARCHAR(255),
    Startdato DATE,
    Sluttdato DATE
);
CREATE TABLE Stol (
    RadNr INT,
    StolNr INT,
    Område VARCHAR(255) CHECK (
        Område IN (
            'Hovedområde',
            'Galleri',
            'Parkett',
            'Balkong'
        )
    ),
    SalNavn VARCHAR(255) NOT NULL,
    PRIMARY KEY (RadNr, StolNr, Område, SalNavn),
    FOREIGN KEY (SalNavn) REFERENCES Teatersal(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Billett (
    RadNr INT NOT NULL,
    StolNr INT NOT NULL,
    Område VARCHAR(255) NOT NULL,
    SalNavn VARCHAR(255) NOT NULL,
    ForestillingNummer INTEGER NOT NULL,
    BillettKjøpNummer INT NOT NULL,
    Billettype VARCHAR(255) NOT NULL,
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    PRIMARY KEY (RadNr, StolNr, Område, SalNavn, ForestillingNummer),
    FOREIGN KEY (BillettKjøpNummer) REFERENCES Billettkjøp(Nummer) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (Billettype, TeaterStykkeNavn) REFERENCES BilletType(Billettype, TeaterStykkeNavn) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (RadNr, StolNr, Område, SalNavn) REFERENCES Stol(RadNr, StolNr, Område, Navn) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN Key (ForestillingNummer) REFERENCES Forestilling(Nummer) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Billettkjøp (
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    ForestillingNummer INTEGER NOT NULL,
    Nummer INTEGER NOT NULL,
    Mobilnummer VARCHAR(20) NOT NULL,
    Dato DATE,
    Tid TIME,
    PRIMARY KEY (Nummer, ForestillingNummer, TeaterStykkeNavn),
    FOREIGN KEY (ForestillingNummer) REFERENCES Forestilling(Nummer) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (Mobilnummer) REFERENCES Kundeprofil(Mobilnummer) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Forestilling (
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    Nummer INTEGER NOT NULL,
    Dato DATE,
    Tid TIME,
    PRIMARY KEY (Nummer, TeaterStykkeNavn),
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE BilletType (
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    Type VARCHAR(255) NOT NULL,
    Pris DECIMAL NOT NULL,
    MinAntall INT NOT NULL,
    PRIMARY KEY (TeaterStykkeNavn, Type),
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Rolle (
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    Navn VARCHAR(255) NOT NULL,
    PRIMARY KEY (TeaterStykkeNavn, Navn),
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Akt (
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    Nummer INTEGER NOT NULL,
    Aktnavn VARCHAR(255) NOT NULL,
    PRIMARY KEY (TeaterStykkeNavn, Nummer),
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE RollePaAkt (
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    Actnummer INT NOT NULL,
    RolleNavn VARCHAR(255) NOT NULL,
    PRIMARY KEY (TeaterStykkeNavn, Actnummer, RolleNavn),
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (RolleNavn) REFERENCES Rolle(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Task (
    Beskrivelse TEXT NOT NULL,
    Identifikator INT NOT NULL,
    Navn VARCHAR(255) NOT NULL,
    PRIMARY KEY (Beskrivelse, Identifikator, Navn),
    FOREIGN KEY (Beskrivelse) REFERENCES Oppgave(Beskrivelse) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (Identifikator) REFERENCES Ansatt(AnsattId) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (Navn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
CREATE TABLE Skuespiller (
    Identifikator INT NOT NULL,
    TeaterStykkeNavn VARCHAR(255) NOT NULL,
    RolleNavn VARCHAR(255) NOT NULL,
    PRIMARY KEY (Identifikator, TeaterStykkeNavn, RolleNavn),
    FOREIGN KEY (Identifikator) REFERENCES Ansatt(AnsattId) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (TeaterStykkeNavn) REFERENCES Theaterstykke(Navn) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (RolleNavn) REFERENCES Rolle(Navn) ON DELETE CASCADE ON UPDATE CASCADE
);
"""
)


# Display the list of tables created in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('Teatersal',),
 ('Theaterstykke',),
 ('Oppgave',),
 ('Oppstilling',),
 ('Ansatt',),
 ('sqlite_sequence',),
 ('Kundeprofil',),
 ('Sesong',),
 ('Stol',),
 ('Billett',),
 ('Billettkjøp',),
 ('Forestilling',),
 ('BilletType',),
 ('Rolle',),
 ('Akt',),
 ('RollePaAkt',),
 ('Task',),
 ('Skuespiller',)]

# Oppgave 1

In [6]:
cursor.execute("INSERT INTO Theaterstykke (Navn) VALUES ('Kongsemnene')")
cursor.execute("INSERT INTO Theaterstykke (Navn) VALUES ('Størst av alt er kjærligheten')")

cursor.execute("INSERT INTO Teatersal (Navn, Kapasitet) VALUES ('Hovedscenen', 524)")
cursor.execute("INSERT INTO Teatersal (Navn, Kapasitet) VALUES ('Gamle scene', 332)")

# Insert into Forestilling for 'Kongsemnene'
dates_kongsemnene = ['2024-02-01', '2024-02-02', '2024-02-03', '2024-02-05', '2024-02-06']
for index, date in enumerate(dates_kongsemnene):
    cursor.execute("INSERT INTO Forestilling (TeaterStykkeNavn, Nummer, Dato, Tid) VALUES ('Kongsemnene', ?, ?, '19:00')", (index+1, date))

# Insert into Forestilling for 'Størst av alt er kjærligheten'
dates_stor = ['2024-02-03', '2024-02-06', '2024-02-07', '2024-02-12', '2024-02-13', '2024-02-14']
for index, date in enumerate(dates_stor):
    cursor.execute("INSERT INTO Forestilling (TeaterStykkeNavn, Nummer, Dato, Tid) VALUES ('Størst av alt er kjærligheten', ?, ?, '18:30')", (index+1, date))

# Insert into BilletTyper for 'Kongsemnene'
BilletTyper_kongsemnene = [
    ('Kongsemnene', 'Ordinær', 450, 0),
    ('Kongsemnene', 'Honnør', 380, 0),
    ('Kongsemnene', 'Student', 280, 0),
    ('Kongsemnene', 'Gruppe 10', 420, 10),
    ('Kongsemnene', 'Gruppe honnør 10', 360, 10)
]
cursor.executemany("INSERT INTO BilletType (TeaterStykkeNavn, Type, Pris, MinAntall) VALUES (?, ?, ?, ?)", BilletTyper_kongsemnene)

# Insert into BilletTyper for 'Størst av alt er kjærligheten'
BilletTyper_stor = [
    ('Størst av alt er kjærligheten', 'Ordinær', 350, 0),
    ('Størst av alt er kjærligheten', 'Honnør', 300, 0),
    ('Størst av alt er kjærligheten', 'Student', 220, 0),
    ('Størst av alt er kjærligheten', 'Barn', 220, 0),
    ('Størst av alt er kjærligheten', 'Gruppe 10', 320, 10),
    ('Størst av alt er kjærligheten', 'Gruppe honnør 10', 270, 10)
]
cursor.executemany("INSERT INTO BilletType (TeaterStykkeNavn, Type, Pris, MinAntall) VALUES (?, ?, ?, ?)", BilletTyper_stor)

# Commit the changes
conn.commit()

In [7]:
# Check the insertion
cursor.execute("SELECT * FROM Theaterstykke")
theaterstykke_data = cursor.fetchall()
cursor.execute("SELECT * FROM Teatersal")
teatersal_data = cursor.fetchall()
cursor.execute("SELECT * FROM Forestilling")
forestilling_data = cursor.fetchall()
cursor.execute("SELECT * FROM BilletType")
BilletTyper_data = cursor.fetchall()

(theaterstykke_data, teatersal_data, forestilling_data, BilletTyper_data)


([('Kongsemnene',), ('Størst av alt er kjærligheten',)],
 [('Hovedscenen', 524), ('Gamle scene', 332)],
 [('Kongsemnene', 1, '2024-02-01', '19:00'),
  ('Kongsemnene', 2, '2024-02-02', '19:00'),
  ('Kongsemnene', 3, '2024-02-03', '19:00'),
  ('Kongsemnene', 4, '2024-02-05', '19:00'),
  ('Kongsemnene', 5, '2024-02-06', '19:00'),
  ('Størst av alt er kjærligheten', 1, '2024-02-03', '18:30'),
  ('Størst av alt er kjærligheten', 2, '2024-02-06', '18:30'),
  ('Størst av alt er kjærligheten', 3, '2024-02-07', '18:30'),
  ('Størst av alt er kjærligheten', 4, '2024-02-12', '18:30'),
  ('Størst av alt er kjærligheten', 5, '2024-02-13', '18:30'),
  ('Størst av alt er kjærligheten', 6, '2024-02-14', '18:30')],
 [('Kongsemnene', 'Ordinær', 450, 0),
  ('Kongsemnene', 'Honnør', 380, 0),
  ('Kongsemnene', 'Student', 280, 0),
  ('Kongsemnene', 'Gruppe 10', 420, 10),
  ('Kongsemnene', 'Gruppe honnør 10', 360, 10),
  ('Størst av alt er kjærligheten', 'Ordinær', 350, 0),
  ('Størst av alt er kjærligheten'

In [8]:
main_area_seats = 504
gallery_area_seats = 520 - 504  # From 504 to 520

# Define the number of seats per row in the main area and the number of rows with 24 seats.
seats_per_row_main = 28
rows_with_24_seats = 2

# Calculate the number of full 28-seat rows
full_rows =18

# Insert seats for the main area
for row in range(1, full_rows + 1):
    for seat_number in range(1, seats_per_row_main + 1):
        seat_number = seat_number + (row-1)*28
        if( 466 < seat_number < 471 ): 
            continue
        if ( 494 < seat_number < 499 ): 
            continue
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, SalNavn) VALUES (?, ?, 'Hovedområde', 'Hovedscenen')",
                       (row, seat_number))

# Insert seats for the gallery area, which has no rows, so we'll consider it as one row with 16 seats.
for seat_number in range(main_area_seats + 1, gallery_area_seats + main_area_seats + 1):
    if( 504 < seat_number < 515): 
        radnr = 1
    else :
        radnr = 2
    cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, SalNavn) VALUES (?, ?, 'Galleri', 'Hovedscenen')",
                   (radnr, seat_number))

# Commit the changes to the database
conn.commit()

In [9]:
# Now let's fetch some of the data to verify the insertions

query = "SELECT * FROM Stol where SalNavn='Hovedscenen' ORDER BY StolNr, Område, RadNr "

cursor.execute(query)
rows = cursor.fetchall()

# Printing the result
print("Seats in the database:")
print("RadNr | StolNr | Område | Teatersal")
for row in rows:
    radnr, stolnr, område, navn = row
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:10} | {navn}")


Seats in the database:
RadNr | StolNr | Område | Teatersal
    1 |      1 | Hovedområde | Hovedscenen
    1 |      2 | Hovedområde | Hovedscenen
    1 |      3 | Hovedområde | Hovedscenen
    1 |      4 | Hovedområde | Hovedscenen
    1 |      5 | Hovedområde | Hovedscenen
    1 |      6 | Hovedområde | Hovedscenen
    1 |      7 | Hovedområde | Hovedscenen
    1 |      8 | Hovedområde | Hovedscenen
    1 |      9 | Hovedområde | Hovedscenen
    1 |     10 | Hovedområde | Hovedscenen
    1 |     11 | Hovedområde | Hovedscenen
    1 |     12 | Hovedområde | Hovedscenen
    1 |     13 | Hovedområde | Hovedscenen
    1 |     14 | Hovedområde | Hovedscenen
    1 |     15 | Hovedområde | Hovedscenen
    1 |     16 | Hovedområde | Hovedscenen
    1 |     17 | Hovedområde | Hovedscenen
    1 |     18 | Hovedområde | Hovedscenen
    1 |     19 | Hovedområde | Hovedscenen
    1 |     20 | Hovedområde | Hovedscenen
    1 |     21 | Hovedområde | Hovedscenen
    1 |     22 | Hovedområde | Hovedsc

In [10]:
# Insert seats for the Galleri area of Gamle scene
galleri_seat_counts = [33, 18, 17]
for row, seats in enumerate(galleri_seat_counts, start=1):
    for seat_number in range(1, seats + 1):
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, SalNavn) VALUES (?, ?, 'Galleri', 'Gamle scene')",
                       (row, seat_number))

# Insert seats for the Balkong area of Gamle scene
balkong_seat_counts = [18, 27, 22, 17]
for row, seats in enumerate(balkong_seat_counts, start=1):
    for seat_number in range(1, seats + 1):
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, SalNavn) VALUES (?, ?, 'Balkong', 'Gamle scene')",
                       (row, seat_number))

# Insert seats for the Parket area of Gamle scene
parket_seat_counts = [18, 16, 17, 18, 18, 17, 18, 17, 17, 14]
for row, seats in enumerate(parket_seat_counts, start=1):
    for seat_number in range(1, seats + 1):
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, SalNavn) VALUES (?, ?, 'Parkett', 'Gamle scene')",
                       (row, seat_number))

# Commit the changes to the database
conn.commit()


In [11]:

query = "SELECT * FROM Stol where SalNavn='Gamle scene' ORDER BY SalNavn, Område, RadNr, StolNr "

cursor.execute(query)
rows = cursor.fetchall()

# Printing the result
print("Seats in the database:")
print("RadNr | StolNr | Område | Teatersal")
for row in rows:
    radnr, stolnr, område, navn = row
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:10} | {navn}")



Seats in the database:
RadNr | StolNr | Område | Teatersal
    1 |      1 | Balkong    | Gamle scene
    1 |      2 | Balkong    | Gamle scene
    1 |      3 | Balkong    | Gamle scene
    1 |      4 | Balkong    | Gamle scene
    1 |      5 | Balkong    | Gamle scene
    1 |      6 | Balkong    | Gamle scene
    1 |      7 | Balkong    | Gamle scene
    1 |      8 | Balkong    | Gamle scene
    1 |      9 | Balkong    | Gamle scene
    1 |     10 | Balkong    | Gamle scene
    1 |     11 | Balkong    | Gamle scene
    1 |     12 | Balkong    | Gamle scene
    1 |     13 | Balkong    | Gamle scene
    1 |     14 | Balkong    | Gamle scene
    1 |     15 | Balkong    | Gamle scene
    1 |     16 | Balkong    | Gamle scene
    1 |     17 | Balkong    | Gamle scene
    1 |     18 | Balkong    | Gamle scene
    2 |      1 | Balkong    | Gamle scene
    2 |      2 | Balkong    | Gamle scene
    2 |      3 | Balkong    | Gamle scene
    2 |      4 | Balkong    | Gamle scene
    2 |      5 | 

# Oppgave 2

In [12]:
# 1. Create a customer profile in 'Kundeprofil'
customer_profile = ("1234567890", "Kari Nordmann", "Bakkegata 3, 1234 Storbyen")
cursor.execute(
    "INSERT INTO Kundeprofil ( Mobilnummer, Navn, Adresse) VALUES (?, ?, ?)",
    customer_profile,
)

# 2. Create a record in 'Billettkjøp'
forestillingsIdQuery = "SELECT Nummer FROM Forestilling WHERE teaterstykkeNavn = 'Størst av alt er kjærligheten' AND dato = '2024-02-03' "
forestillingsId = cursor.execute(forestillingsIdQuery).fetchone()[0]

ticket_purchase = (
    1,
    forestillingsId,
    "Størst av alt er kjærligheten",
    "1234567890",
    "2024-02-03",
    "18:30",
)
cursor.execute(
    "INSERT INTO Billettkjøp (Nummer, ForestillingNummer,  TeaterStykkeNavn, Mobilnummer,  dato, tid) VALUES (?,?, ?, ?, ?, ?)",
    ticket_purchase,
)
conn.commit()

In [13]:
# 3. Buy specified seats
# Define the seating layout as provided, '1' for purchased, '0' for not purchased
seating_layout = """
00000000000000000
000000000000000000
000001100000000000000000000000000
Galleri
00000000000000000
0000000000000000000000
110000000000000000000000000
0000000000000000000000000000
Balkong
00000001000000
00000001000000000
00000001000000000
000000011000000000
00000001000000000
000000011000000000
000110011000000000
00011000100000000
1110000010000000
110000011000000000
Parkett
"""
# Process the seating layout
rows = seating_layout.strip().split("\n")
# reverse the rows
rows = rows[::-1]
current_area = ""
for row in rows:
    if row in ["Galleri", "Balkong", "Parkett"]:
        current_area = row
        row_counter = 1
    else:
        for seat_counter, seat in enumerate(row, start=1):
            if seat == "1":

                cursor.execute(
                    "INSERT INTO Billett (RadNr, StolNr, Område, SalNavn, ForestillingNummer, BillettKjøpNummer, Billettype, TeaterStykkeNavn ) VALUES (?, ?, ?, 'Gamle scene', ? ,?, ? , ?)",
                    (
                        row_counter,
                        seat_counter,
                        current_area,
                        forestillingsId,
                        1,
                        "Ordinær",
                        "Størst av alt er kjærligheten",
                    ),
                )
        row_counter += 1

# Commit the changes and close the connection
conn.commit()

In [14]:

query = "SELECT RadNr, StolNr, Område, SalNavn FROM Billett ORDER BY SalNavn, Område, RadNr DESC, StolNr DESC"

cursor.execute(query)
bought_seats = cursor.fetchall()

# Printing the result
print("Bought Seats:")
print("RadNr | StolNr | Område       | Teatersal")
for seat in bought_seats:
    radnr, stolnr, område, navn = seat
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:12} | {navn}")



Bought Seats:
RadNr | StolNr | Område       | Teatersal
    2 |      2 | Balkong      | Gamle scene
    2 |      1 | Balkong      | Gamle scene
    1 |      7 | Galleri      | Gamle scene
    1 |      6 | Galleri      | Gamle scene
   10 |      8 | Parkett      | Gamle scene
    9 |      8 | Parkett      | Gamle scene
    8 |      8 | Parkett      | Gamle scene
    7 |      9 | Parkett      | Gamle scene
    7 |      8 | Parkett      | Gamle scene
    6 |      8 | Parkett      | Gamle scene
    5 |      9 | Parkett      | Gamle scene
    5 |      8 | Parkett      | Gamle scene
    4 |      9 | Parkett      | Gamle scene
    4 |      8 | Parkett      | Gamle scene
    4 |      5 | Parkett      | Gamle scene
    4 |      4 | Parkett      | Gamle scene
    3 |      9 | Parkett      | Gamle scene
    3 |      5 | Parkett      | Gamle scene
    3 |      4 | Parkett      | Gamle scene
    2 |      9 | Parkett      | Gamle scene
    2 |      3 | Parkett      | Gamle scene
    2 |      2 | Par

In [15]:
# Define the seating pattern for the tickets to be purchased
seating_pattern = [
    "000000000000000000xxxx000000",
    "000000000000000000xxxx000000",
    "0000000000100000000000000000",
    "0000011100100000000000000000",
    "0000000000100000000000000000",
    "1110000000100000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000001100000000000000",
    "0000000000000110000000000000",
    "0000000000000011000000000000",
    "0000000000000001100000000000",
    "0000000001111111111000000000",
    "1111111111111111111000000000",
    "0000000000111111111111111111",
]


# Identify the seats to be purchased
def find_seats_to_purchase(seating_pattern):
    seats_to_purchase = []

    seating_pattern = seating_pattern[::-1]
    for row_number, row in enumerate(seating_pattern, start=1):
        for seat_number, seat in enumerate(row, start=1):
            seatnumber_sum = seat_number + (row_number - 1) * 28
            if seat == "1":
                seats_to_purchase.append((row_number, seatnumber_sum))
    return seats_to_purchase


# Seats to be purchased
seats_to_purchase = find_seats_to_purchase(seating_pattern)
print(seats_to_purchase)

performance_date = "2024-02-03"
get_ForestillingNummer = "SELECT Nummer FROM Forestilling WHERE teaterstykkeNavn = 'Kongsemnene' AND dato = ?"
cursor.execute(get_ForestillingNummer, (performance_date,))
forestilling_id = cursor.fetchone()[0]

print(forestilling_id)

# create billettkjøp
cursor.execute(
    "INSERT INTO Billettkjøp (TeaterStykkeNavn, ForestillingNummer, Nummer, MobilNummer, dato, tid) VALUES (?, ?,?,?, ?, ?)",
    ("Kongsemnene", forestilling_id, 1, "1234567890", "2024-02-03", "19:00"),
)


# Insert tickets for each selected seat
forestilling_id = (
    1  # Assuming an ID for the 'Forestilling'. Replace with the correct ID.
)

for seat in seats_to_purchase:
    radnr, stolnr = seat
    omrade = "Hovedområde" if radnr <= len(seating_pattern) else "Galleri"
    sal_navn = "Hovedscenen"
    billettype = "Ordinær"

    # Prepare the SQL command
    sql_command = """
        INSERT INTO Billett 
        (RadNr, StolNr, Område, salNavn, ForestillingNummer, BillettKjøpNummer, Billettype, TeaterStykkeNavn)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """
    values = (
        radnr,
        stolnr,
        omrade,
        sal_navn,
        forestilling_id,
        1,
        billettype,
        "Kongsemnene",
    )

    # Execute the command
    cursor.execute(sql_command, values)

# # Commit the changes
conn.commit()

[(1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (2, 29), (2, 30), (2, 31), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (2, 39), (2, 40), (2, 41), (2, 42), (2, 43), (2, 44), (2, 45), (2, 46), (2, 47), (3, 66), (3, 67), (3, 68), (3, 69), (3, 70), (3, 71), (3, 72), (3, 73), (3, 74), (3, 75), (4, 100), (4, 101), (5, 127), (5, 128), (6, 154), (6, 155), (7, 181), (7, 182), (13, 337), (13, 338), (13, 339), (13, 347), (14, 375), (15, 398), (15, 399), (15, 400), (15, 403), (16, 431)]
3


In [16]:
query = "SELECT RadNr, StolNr, Område, SalNavn FROM Billett where TeaterStykkeNavn = 'Kongsemnene' ORDER BY SalNavn, Område, RadNr DESC, StolNr DESC"

cursor.execute(query)
bought_seats = cursor.fetchall()

# Printing the result
print("Bought Seats:")
print("RadNr | StolNr | Område       | Teatersal")
for seat in bought_seats:
    radnr, stolnr, område, navn = seat
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:12} | {navn}")

Bought Seats:
RadNr | StolNr | Område       | Teatersal
   16 |    431 | Hovedområde  | Hovedscenen
   15 |    403 | Hovedområde  | Hovedscenen
   15 |    400 | Hovedområde  | Hovedscenen
   15 |    399 | Hovedområde  | Hovedscenen
   15 |    398 | Hovedområde  | Hovedscenen
   14 |    375 | Hovedområde  | Hovedscenen
   13 |    347 | Hovedområde  | Hovedscenen
   13 |    339 | Hovedområde  | Hovedscenen
   13 |    338 | Hovedområde  | Hovedscenen
   13 |    337 | Hovedområde  | Hovedscenen
    7 |    182 | Hovedområde  | Hovedscenen
    7 |    181 | Hovedområde  | Hovedscenen
    6 |    155 | Hovedområde  | Hovedscenen
    6 |    154 | Hovedområde  | Hovedscenen
    5 |    128 | Hovedområde  | Hovedscenen
    5 |    127 | Hovedområde  | Hovedscenen
    4 |    101 | Hovedområde  | Hovedscenen
    4 |    100 | Hovedområde  | Hovedscenen
    3 |     75 | Hovedområde  | Hovedscenen
    3 |     74 | Hovedområde  | Hovedscenen
    3 |     73 | Hovedområde  | Hovedscenen
    3 |     72 | Hov

# Resten av oppgave 1
Vi tok først det som var nødvendig for å kjøpe billetter og her kommer alle skuespiller, akter, roller og ansatte.

In [17]:
# cursor.execute("INSERT INTO Akt (teaterstykkeNavn, nummer, aktnavn) VALUES ('Kongsemnene', 2, '2',)")
# insert act 1 to 5

acts = [
    ("Kongsemnene", 1, "1"),
    ("Kongsemnene", 2, "2"),
    ("Kongsemnene", 3, "3"),
    ("Kongsemnene", 4, "4"),
    ("Kongsemnene", 5, "5"),
]
cursor.executemany(
    "INSERT INTO Akt (teaterstykkeNavn, nummer, aktnavn) VALUES (?, ?, ?)", acts
)

rols = [
("Håkon Håkonssønn",),
("Dagfinn Bonde",),
("Jatgeir Skald",),
("Sigrid",),
("Ingeborg",),
("Guttorm Ingesson",),
("Skule Jarl",),
("Inga frå Vartejg",),
("Paal Flida",),
("Gregorius Jonsson",),
("Margrete",),
("Biskop Nikolas",),
("Peter",),
("Trønder",),
("Bård Bratte",),
]
cursor.executemany(
    "INSERT INTO Rolle (teaterstykkeNavn, navn) VALUES ('Kongsemnene', ?)", rols
)


In [18]:

# SQL query for inserting data into RollePaAkt table
sql_query = """
INSERT INTO RollePaAkt (teaterstykkeNavn, Actnummer, rolleNavn)
VALUES
('Kongsemnene', 1, 'Håkon Håkonson'),
('Kongsemnene', 2, 'Håkon Håkonson'),
('Kongsemnene', 3, 'Håkon Håkonson'),
('Kongsemnene', 4, 'Håkon Håkonson'),
('Kongsemnene', 5, 'Håkon Håkonson'),

('Kongsemnene', 1, 'Dagfinn Bonde'),
('Kongsemnene', 2, 'Dagfinn Bonde'),
('Kongsemnene', 3, 'Dagfinn Bonde'),
('Kongsemnene', 4, 'Dagfinn Bonde'),
('Kongsemnene', 5, 'Dagfinn Bonde'),

('Kongsemnene', 4, 'Jatgeir Skald'),

('Kongsemnene', 1, 'Sigrid'),
('Kongsemnene', 2, 'Sigrid'),
('Kongsemnene', 5, 'Sigrid'),

('Kongsemnene', 4, 'Ingeborg'),

('Kongsemnene', 1, 'Guttorm Ingesson'),

('Kongsemnene', 1, 'Skule Jarl'),
('Kongsemnene', 2, 'Skule Jarl'),
('Kongsemnene', 3, 'Skule Jarl'),
('Kongsemnene', 4, 'Skule Jarl'),
('Kongsemnene', 5, 'Skule Jarl'),

('Kongsemnene', 1, 'Inga fra Vartejg'),
('Kongsemnene', 3, 'Inga fra Vartejg'),

('Kongsemnene', 1, 'Paal Flida'),
('Kongsemnene', 2, 'Paal Flida'),
('Kongsemnene', 3, 'Paal Flida'),
('Kongsemnene', 4, 'Paal Flida'),
('Kongsemnene', 5, 'Paal Flida'),

('Kongsemnene', 1, 'Fru Ragnhild'),
('Kongsemnene', 5, 'Fru Ragnhild'),

('Kongsemnene', 1, 'Gregorius Jonsson'),
('Kongsemnene', 2, 'Gregorius Jonsson'),
('Kongsemnene', 3, 'Gregorius Jonsson'),
('Kongsemnene', 4, 'Gregorius Jonsson'),
('Kongsemnene', 5, 'Gregorius Jonsson'),

('Kongsemnene', 1, 'Margrete'),
('Kongsemnene', 2, 'Margrete'),
('Kongsemnene', 3, 'Margrete'),
('Kongsemnene', 4, 'Margrete'),
('Kongsemnene', 5, 'Margrete'),

('Kongsemnene', 1, 'Biskop Nikolas'),
('Kongsemnene', 2, 'Biskop Nikolas'),
('Kongsemnene', 3, 'Biskop Nikolas'),

('Kongsemnene', 3, 'Peter'),
('Kongsemnene', 4, 'Peter'),
('Kongsemnene', 5, 'Peter');
"""

cursor.executescript(sql_query)


In [19]:

# Insert values into Ansatt table
ansatt_data = [
    (1, 'epost1@example.com', 'Arturo Scotti', 'Active'),
    (2, 'epost2@example.com', 'Ingunn Beate Strige Øyen', 'Active'),
    (3, 'epost3@example.com', 'Hans Petter Nilsen', 'Active'),
    (4, 'epost4@example.com', 'Madeleine Brandtzæg Nilsen', 'Active'),
    (5, 'epost5@example.com', 'Synnøve Fossum Eriksen', 'Active'),
    (6, 'epost6@example.com', 'Emma Caroline Deichmann', 'Active'),
    (7, 'epost7@example.com', 'Thomas Jensen Takyi', 'Active'),
    (8, 'epost8@example.com', 'Per Bogstad Gulliksen', 'Active'),
    (9, 'epost9@example.com', 'Isak Holmen Sørensen', 'Active'),
    (10, 'epost10@example.com', 'Fabian Heidelberg Lunde', 'Active'),
    (11, 'epost11@example.com', 'Emil Olafsson', 'Active'),
    (12, 'epost12@example.com', 'Snorre Ryen Tøndel', 'Active'),
]

# Insert data into Ansatt table
cursor.executemany("""
    INSERT INTO Ansatt (ansattId, epost, navn, ansattstatus)
    VALUES (?, ?, ?, ?)
""", ansatt_data)

# Commit the changes to Ansatt table
conn.commit()

skuespiller_data = [
    (1, 'Kongsemnene', 'Håkon Håkonson'),
    (2, 'Kongsemnene', 'Inga fra Vartejg'),
    (3, 'Kongsemnene', 'Skule Jarl'),
    (4, 'Kongsemnene', 'Fru Ragnhild'),
    (5, 'Kongsemnene', 'Margrete'),
    (6, 'Kongsemnene', 'Sigrid'),
    (6, 'Kongsemnene', 'Ingeborg'),
    (7, 'Kongsemnene', 'Biskop Nikolas'),
    (8, 'Kongsemnene', 'Gregorius Jonsson'),
    (9, 'Kongsemnene', 'Paal Flida'),
    (9, 'Kongsemnene', 'Trønder'),
    (10, 'Kongsemnene', 'Baard Bratte'),
    (10, 'Kongsemnene', 'Trønder'),
    (11, 'Kongsemnene', 'Jatgeir Skald'),
    (11, 'Kongsemnene', 'Dagfinn Bonde'),
    (12, 'Kongsemnene', 'Peter'),
]

# Insert data into Skuespiller table
cursor.executemany("""
    INSERT INTO Skuespiller (Identifikator, TeaterStykkeNavn, RolleNavn)
    VALUES (?, ?, ?)
""", skuespiller_data)

# commit
conn.commit()


In [20]:
# lag en sessong som heter vinter 2024
cursor.execute("INSERT INTO Sesong (navn, startdato, sluttdato) VALUES ('Vinter 2024', '2024-01-01', '2024-03-31')")

# lag oppstilling mellom gamle scene og storst av alt er kjærligheten 
cursor.execute("INSERT INTO Oppstilling (SesongId, TeaterStykkeNavn, SalNavn) VALUES (1, 'Størst av alt er kjærligheten', 'Gamle scene')")

# lag en oppstillig for kongsemnene på hovedscenen
cursor.execute("INSERT INTO Oppstilling (SesongId, TeaterStykkeNavn, SalNavn) VALUES (1, 'Kongsemnene', 'Hovedscenen')")
# commit
conn.commit()


# Oppgave 3
Her skal du kjøpe 9 voksenbilletter til forestillingen for Størst av alt er
kjærligheten 3. februar, hvor det er 9 ledige billetter og hvor stolene er på
samme rad. Stolene trenger ikke være ved siden av hverandre. Vi ønsker å få
summert hva det koster å kjøpe disse billettene,

In [21]:
cursor.execute("SELECT * FROM Kundeprofil")
customer = cursor.fetchone()
customer

date = "2024-02-03"

query = "SELECT Nummer FROM Forestilling WHERE teaterstykkeNavn = 'Størst av alt er kjærligheten' AND dato = ?"
cursor.execute(query, (date,))
ForestillingNummer = cursor.fetchone()[0]

# billettkjøp
cursor.execute(
    "INSERT INTO Billettkjøp (TeaterStykkeNavn,ForestillingNummer, Nummer, Mobilnummer, dato, tid) VALUES (?, ?,?,?, ?, ?)",
    (
        "Størst av alt er kjærligheten",
        ForestillingNummer,
        2,
        "1234567890",
        date,
        "18:30",
    ),
)
conn.commit()

# billett
plasser = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# kjøp 9 plasser rad 1  parkett
for plass in plasser:
    cursor.execute(
        "INSERT INTO Billett (RadNr, StolNr, Område, salNavn, ForestillingNummer, BillettKjøpNummer, Billettype, TeaterStykkeNavn ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        (
            2,
            plass,
            "Galleri",
            "Gamle scene",
            ForestillingNummer,
            2,
            "Ordinær",
            "Størst av alt er kjærligheten",
        ),
    )
conn.commit()

In [22]:
# get ticket for forestilling 6 and billettkjøp 2 and join them with ticket type and sum the price

query = """
SELECT SUM(BilletType.pris)
FROM Billett
JOIN BilletType
ON Billett.Billettype = BilletType.type
AND Billett.TeaterStykkeNavn = BilletType.TeaterStykkeNavn
WHERE Billett.ForestillingNummer = ?
AND Billett.BillettKjøpNummer = 2
"""
cursor.execute(query, (ForestillingNummer, ))
tickets = cursor.fetchall()
print(f'Billettene koster {tickets[0][0]} kr')




Billettene koster 3150 kr


# Oppgave 4
Her skal du implementere et Pythonprogram (med bruk av SQL) som tar inn
en dato og skriver ut hvilke forestillinger som finnes på denne datoen og lister
opp hvor mange billetter (dvs. stoler) som er solgt. Ta også med forestillinger
hvor det ikke er solgt noen billetter.

In [23]:

def hent_forestilling_med_dato(date):
    # get forestilling with date and number of tickets sold
    query = """
    SELECT 
    F.TeaterStykkeNavn,
    F.Nummer as ForestillingNummer,
    F.Dato,
    F.Tid,
    count(B.ForestillingNummer) as AntallBilletter
    FROM Forestilling F
    LEFT JOIN Billett B
    ON F.Nummer = B.ForestillingNummer
    AND F.TeaterStykkeNavn = B.TeaterStykkeNavn
    Group BY F.TeaterStykkeNavn, F.Nummer
    HAVING F.Dato = ?
    """
    
    cursor.execute(query, (date,))
    # cursor.execute(query)
    forestillinger = cursor.fetchall()
    headers = [description[0] for description in cursor.description]
    return forestillinger, headers


# for forestilling in hent_forestilling_med_dato("2024-02-03"):
# print(f'{forestilling[1]} går den {forestilling[2]} kl {forestilling[3]}')

# forestillings_dato = input("Oppagve 4: Skriv inn datoen for forestillingen (YYYY-MM-DD): ")
forestillings_dato = "2024-02-03"
forestilling_data, headers = hent_forestilling_med_dato(forestillings_dato)
print(tabulate(forestilling_data, headers=headers , tablefmt="grid"))

+-------------------------------+----------------------+------------+-------+-------------------+
| TeaterStykkeNavn              |   ForestillingNummer | Dato       | Tid   |   AntallBilletter |
+===============================+======================+============+=======+===================+
| Størst av alt er kjærligheten |                    1 | 2024-02-03 | 18:30 |                36 |
+-------------------------------+----------------------+------------+-------+-------------------+
| Kongsemnene                   |                    3 | 2024-02-03 | 19:00 |                 0 |
+-------------------------------+----------------------+------------+-------+-------------------+


# oppgave 5
Vi ønsker å lage et query i SQL som finner hvilke (navn på) skuespillere som
opptrer i de forskjellige teaterstykkene. Skriv ut navn på teaterstykke, navn på
skuespiller og rolle.

In [24]:
def hent_skuespillere_og_roller():
    query = """
    SELECT Rolle.teaterstykkeNavn, Ansatt.navn, Rolle.navn
    FROM Skuespiller
    JOIN Rolle
    ON Skuespiller.RolleNavn = Rolle.navn
    JOIN Ansatt on Skuespiller.Identifikator = ansatt.ansattId
    AND Skuespiller.TeaterStykkeNavn = Rolle.teaterstykkeNavn
    """
    cursor.execute(query)
    skuespillere = cursor.fetchall()
    return skuespillere

Roller = hent_skuespillere_og_roller()
print(tabulate(Roller, headers= ["Teaterstykke", "Skuespiller", "Rolle"], tablefmt="grid"))

+----------------+-------------------------+-------------------+
| Teaterstykke   | Skuespiller             | Rolle             |
+================+=========================+===================+
| Kongsemnene    | Hans Petter Nilsen      | Skule Jarl        |
+----------------+-------------------------+-------------------+
| Kongsemnene    | Synnøve Fossum Eriksen  | Margrete          |
+----------------+-------------------------+-------------------+
| Kongsemnene    | Emma Caroline Deichmann | Sigrid            |
+----------------+-------------------------+-------------------+
| Kongsemnene    | Emma Caroline Deichmann | Ingeborg          |
+----------------+-------------------------+-------------------+
| Kongsemnene    | Thomas Jensen Takyi     | Biskop Nikolas    |
+----------------+-------------------------+-------------------+
| Kongsemnene    | Per Bogstad Gulliksen   | Gregorius Jonsson |
+----------------+-------------------------+-------------------+
| Kongsemnene    | Isak H

# oppgave 6
Vi ønsker å lage et query i SQL som finner hvilke forestillinger som har solgt
best. Skriv ut navn på forestilling og dato og antall solgte plasser sortert på
antall plasser i synkende rekkefølge.

In [25]:

# hent alle billetter join med billetttype og groupe by ForestillingNummer og tell antall
# det er viktig at vi først grupere etter teaterstykke og forestillingnummer og så teller antall 
# sql gruperer etter den etter tur. Vi vil ikke grupere forestillinger med samme nummer til forskjellige teaterstykker
query = """
SELECT Forestilling.TeaterStykkeNavn, Forestilling.dato , COUNT(Billett.ForestillingNummer) as 'Antall solgte plasser'
FROM Billett
JOIN BilletType ON Billett.Billettype = BilletType.type AND Billett.TeaterStykkeNavn = BilletType.TeaterStykkeNavn
JOIN Forestilling ON (Billett.ForestillingNummer = Forestilling.Nummer AND Billett.TeaterStykkeNavn = Forestilling.TeaterStykkeNavn)
Group by Billett.TeaterStykkeNavn, Billett.ForestillingNummer
"""

cursor.execute(query)
forestillinger = cursor.fetchall()
print(tabulate(forestillinger, headers= ["Teaterstykke", "Dato", "Antall solgte plasser"], tablefmt="grid"))


+-------------------------------+------------+-------------------------+
| Teaterstykke                  | Dato       |   Antall solgte plasser |
+===============================+============+=========================+
| Kongsemnene                   | 2024-02-01 |                      65 |
+-------------------------------+------------+-------------------------+
| Størst av alt er kjærligheten | 2024-02-03 |                      36 |
+-------------------------------+------------+-------------------------+


# oppgave 7
Du skal lage et Pythonprogram (og SQL) som tar et skuespillernavn og finner
hvilke skuespilllere de har spilt med i samme akt. Skriv ut navn på begge og
hvilket skuespill det skjedde.

In [26]:
# Du skal lage et Pythonprogram (og SQL) som tar et skuespillernavn og finner
# hvilke skuespilllere de har spilt med i samme akt. Skriv ut navn på begge og
# hvilket skuespill det skjedde.
# jeg anter at det menes act og teaterstykke når det står skuespill

# finn alle akter som skuespilleren har spilt i
def hent_akter(skuespiller):
    query = """
    SELECT DISTINCT RPA.Actnummer, S1.TeaterStykkeNavn
    FROM Skuespiller S1
    JOIN RollePaAkt RPA ON S1.TeaterStykkeNavn = RPA.teaterstykkeNavn AND S1.RolleNavn = RPA.rolleNavn
    JOIN Ansatt A1 on S1.Identifikator = A1.ansattId
    WHERE A1.navn = ?
    """
    cursor.execute(query, (skuespiller,))
    akter = cursor.fetchall()
    return akter


def hent_skuespillere_i_samme_akt(skuespiller):
    akter = hent_akter(skuespiller)
    print(tabulate
          (akter, headers= ["Akt", "Teaterstykke"], tablefmt="grid"))
    query = """
    SELECT
        S1.teaterstykkenavn,
        S2.navn as Actor,
        S2.RolleNavn,
        A1.navn as CoActor,
        S1.rolleNavn,
        RPA.Actnummer
    FROM
        Skuespiller S1
        join Ansatt A1 on S1.Identifikator = A1.ansattId
        JOIN RollePaAkt RPA ON S1.TeaterStykkeNavn = RPA.teaterstykkeNavn AND S1.RolleNavn = RPA.rolleNavn
        JOIN (
            SELECT S2.TeaterStykkeNavn, S2.RolleNavn, A2.navn, RPA2.Actnummer, A2.AnsattId
            FROM Skuespiller S2
            JOIN Ansatt A2 on S2.Identifikator = A2.ansattId
            JOIN RollePaAkt RPA2 ON S2.TeaterStykkeNavn = RPA2.teaterstykkeNavn AND S2.RolleNavn = RPA2.rolleNavn
            WHERE A2.navn = ?
        ) as S2
        ON S1.TeaterStykkeNavn = S2.TeaterStykkeNavn AND RPA.Actnummer = S2.Actnummer And A1.AnsattId != S2.AnsattId
        order by S1.rolleNavn

    """
    # cursor.execute(query, (skuespiller,))
    cursor.execute(query, (skuespiller,))
    
    # print column names
    print()
    skuespillere = cursor.fetchall()
    return skuespillere

# get all actors names 
query = "SELECT navn FROM Ansatt"
cursor.execute(query)
actors = cursor.fetchall()

# skuespillere = hent_skuespillere_i_samme_akt("Thomas Jensen Takyi")
skuespillere = hent_skuespillere_i_samme_akt("Ingunn Beate Strige Øyen")
print("––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––")

headers = [description[0] for description in cursor.description]
print(tabulate(skuespillere, headers=headers, tablefmt="grid"))
# print(tabulate(skuespillere, headers=["Skuespiller 1", "Skuespiller 2", "Teaterstykke", "Akt"], tablefmt="grid"))

+-------+----------------+
|   Akt | Teaterstykke   |
+=======+================+
|     1 | Kongsemnene    |
+-------+----------------+
|     3 | Kongsemnene    |
+-------+----------------+

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
+--------------------+--------------------------+------------------+----------------------------+-------------------+-------------+
| TeaterStykkeNavn   | Actor                    | RolleNavn        | CoActor                    | RolleNavn         |   Actnummer |
+====================+==========================+==================+============================+===================+=============+
| Kongsemnene        | Ingunn Beate Strige Øyen | Inga fra Vartejg | Thomas Jensen Takyi        | Biskop Nikolas    |           1 |
+--------------------+--------------------------+------------------+----------------------------+-------------------+-------------+
| Kongsemnene        | Ingunn Beate Strige Øyen | Inga fra Vartejg | Thomas J